# F-strings: A format system to rule them all

#### Juan Diego Godoy Robles, PyConES 2019, Alicante

La interpolación de cadenas permite embeber código en cadenas de texto usando una sintaxis mínima y limitando al máximo los posibles errores.

[Wikipedia](https://en.wikipedia.org/wiki/String_interpolation): es el proceso de evaluación de una cadena de de texto literal que contiene al menos un placeholder y que devuelve un resultado en el que estos tokens son reemplazados con los valores resultantes.

In [ ]:
from datetime import date, datetime, timedelta
talk = 'f-strings'
talk_date = date(2019, 10 , 5)
me = 'juan diego'
minutes_left = 20

print (
  f'Ey Pythonists folks!, today is {talk_date:%A %d %B of %Y}.\n'
  f'I\'m {me.title()}, wellcome to this awesome {talk!r} talk.\nYou\'ll be free '
  f'at {datetime.now() + timedelta(minutes=minutes_left):%H:%M} \U0001f44d'
)

Esta feature (presente previamente en numerosos lenguajes), fue propuesta en la [PEP-0498](https://www.python.org/dev/peps/pep-0498/), aprobada no sin su buena dosis de [polémica](https://www.reddit.com/r/Python/comments/3k6qi8/pep_498_approved/) y constituye la base para esta charla.

## ¿Cómo se había resuelto esto antes?

Existen tres metodos, que siguen absolutamente vigentes ya que las` f-strings` no suponen la  _deprecración_ de ninguno de los anteriores.

### Printf style formatting

Los objetos tipo String disponen de un operador de [interpolación](https://docs.python.org/3/library/stdtypes.html#printf-style-string-formatting) similar al `sprintf` en **C**.

Si el proceso de _format_ requiere más de un argumento, el valor deberá ser una _tupla_ cuyo número de elementos deberá coincidir con los especificados en la cadena a formatear, o un objeto que mapee _key-value_ (por ejemplo un diccionario)

In [ ]:
'%s %s' % ('Old Fashioned', 'formatting')

In [ ]:
'%(style)s %(action)s' % {'style': 'Old Fashioned', 'action': 'formatting'}

Una de sus desventajas es que necesita especificar el tipo de conversion:

In [ ]:
'%' % 'hello'

Solo permite el formateo de enteros, dobles o cadenas. Cualquier otro tipo debe ser convertido a los anteriores antes de aplicar el formato (quizás esto no suponga un problema serio ya que casi cualquier objeto tiene implementado el metodo __str__, __repr__).

Por otro lado existe un problema conocido al pasarle una tupla con mas de un elemento:

In [ ]:
'%s' % ('%-formating sucks',  'much')

Podemos evitarlo programando de forma defensiva:

In [ ]:
'%s' % (('%-formating sucks',  'much'),)

O conociendo de antemano el número de elementos lo cual es muy poco flexible:

In [ ]:
'%s %s' % ('%-formating sucks',  'much')

### Format String Syntax

Mi [método](https://docs.python.org/3/library/string.html#formatstrings) favorito hasta la llegada de las `f-strings`, de hecho  estas ultimas reutilizan gran parte de la sintaxis y los mecanismos de format pero también son mucho más _verbose_.

In [ ]:
awesome_conf = 'PyConES'

'This {awesome_conf} is really awesome.'.format(awesome_conf=awesome_conf)

Incluso si lo simplificamos al máximo, podemos ver como la variable queda un poco desconectada del contexto lo cual se hace mucho más evidente al compararlo con el mismo ejemplo usando `f-strings`:

In [ ]:
awesome_conf = 'PyConES'

print('This {} is really awesome.'.format(awesome_conf))

print(f'This {awesome_conf} is really awesome.')


### Template strings

Las [templates ](https://docs.python.org/3/library/string.html#template-strings) se crearon como alternativa al operador de interpolación, que como hemos visto es muy propenso a errores, con el único [objetivo en mente](https://www.python.org/dev/peps/pep-0292/) de simplificar los modos de formatear (en este caso el termino más apropiado sería substituir) cadenas, con el _trade-off_ de sacrificar la expresividad.

In [ ]:
from string import Template
Template('Hello from $this').substitute(this='Template')

No soportan el protocolo *format*, por lo que no es posible realizar conversiones.

El bajo rendimiento y su poca flexibilidad son otros de sus putos débiles.

## F-strings

### ¿Qué son?

Una forma de embeber en objetos tipo *String* expresiones que se evalúan en tiempo de ejecución.

In [ ]:
type(f'{type}')

### ¿Qué cambios suponen? 

**Ninguno**, los métodos anteriores no han sido deprecados.

### ¿Qué ventajas nos aportan?

Dos fundamentalmente: 

1. Claridad: Sin duda el principal aporte de las `F-strings` es la mejora de la legibilidad  de nuestro código, juzguen ustedes mismos:

In [ ]:
ways_2_formatx2 = 6
awesome_lang = 'Python'

print(
    'In %s we have %d ways to do our formatting'
    % (awesome_lang, int(ways_2_formatx2/2))
)
print(
    'In {awesome_lang:s} we have {ways_2_format:d} ways to do our formatting'.format(
        awesome_lang=awesome_lang, ways_2_format=(int(ways_2_formatx2/2))
    )
)
print (f'In {awesome_lang} we have {int(ways_2_formatx2/2)} ways to do our formatting')

2. Rendimiento: una `F-string` en primer lugar se evalúa la expresión en tiempo de ejecución y después se combina con la porción literal para devolver la cadena final. No existe ningún otro requerimiento, esto las hace muy rápidas y eficientes: se podría decir que al mismo nivel que la interpolación y superiores a `format`:

In [ ]:
import dis

def foo():
    x = 42
    y = 99
    return '{} + {} = {}'.format(x, y, x + y)

dis.dis(foo)

Con `LOAD_ATTR`  python referencia a la función format a la que posteriormente llama con `CALL_FUNCTION`  lo cual resulta más pesado que el tratamiento con `F-strings` libre de este _overhead_:

In [ ]:
import dis

def foo():
    x = 42
    y = 99
    return f'{x} + {y} = {x + y}'

dis.dis(foo)

Podemos comprobarlo en la práctica con una sencilla prueba:

In [ ]:
import timeit

print(timeit.timeit("""name='PyConES';year=2019;f'{name} - {year}'""", number=10000000))
print(timeit.timeit("""name='PyConES';year=2019;'%s - %d' % (name, year)""", number=10000000))
print(timeit.timeit("""name='PyConES';year=2019;'{} - {}'.format(name, year)""", number=10000000))

¿Cómo funciona?

En la  **compilación** solo se podrá detectar errores de sintaxis, por ejemplo si nos dejamos alguna llave (`{` o `}`)  _coja_.

Al **ejecutar**, la expresión se evaluará en el contexto en el que aparezca la `F-string`, por lo que tendrá pleno acceso a las *variables locales y globales*.

Estos dos `print` son totalmente equivalentes:

In [ ]:
def hi():
    return 'Hello'

print(f'{hi()} world!')
print(str(hi())+ ' world!')

### ¿Cómo se usan?

Debemos formar una cadena literal a la que se le antepone el prefijo `f` o `F` (ambos son equivalentes).

Por lo demás, su tratamiento es equivalente al de cualquier otra cadena, por ejemplo el carácter que inicia el entrecomillado debe ser igual al que lo finaliza.

Una vez _tokenizado_, una `F-string` se descompone en cadenas literales y expresiones, estas últimas deben de contenerse entre llaves: `{expr}`.

Para escapar una llave, necesitaremos doblarla `{{` o `}}`.

El carácter de escape `\` no está permitido dentro de una expresión, este inconveniente puede ser solventado cambiando el carácter de entrecomillado o usando el _triple quoting_.

Opcionalmente y como última parte de una expresión se puede especificar un tipo de conversión, con un funcionamiento análogo a `format`: `!s`  llama a `str()`, `!r` a `repr()` y `!a` a `ascii()`.

Añadir que se pueden usar *especificadores de formato*, en cuyo caso una vez evaluada la expresión se `parearan` al método `__format__` del objeto resultante para que sean interpretado (equivalente a `format`).

In [ ]:
from datetime import datetime
import decimal

width = 6
precision = 4
hora = decimal.Decimal('13.29999999999999999999')

f'''Playing with {{ {" f-strings '-) ".upper()!s:-^20} }} {datetime.now():%Y}{hora:{width}.{precision}}'''

También podemos usar `F-strings` en *modo raw*, añadiendo el prefijo `r` o `R` . De esta forma el carácter de escape `\` no será interpretado.

In [ ]:
import re

re.search(fr'=\s*{20 * 2}', 'sum=  40')

## Fun with F-strings

### Objects 

In [ ]:
import datetime

class Talk:
    def __init__(self, title, conference, date):
        self.title = title
        self.conference = conference
        self.date = date

    def __str__(self):
        return f'{self.title} ({self.conference} {self.date:%Y})'

    def __repr__(self):
        return f'{self.conference}: Today is {self.date:%A %d %B} Wellcome to {self.title!r}'


my_talk = Talk('f-strings', 'PyConES', datetime.date(2019, 10, 5))
print(f'''{Talk('f-strings', 'PyConES', datetime.date(2019, 10 , 5))!r}''')
print(f'{my_talk}')

### Exceptions

In [ ]:
try:
    print(non_existent)
except Exception as err:
    print(f'an error hapenned: {err}')

### Multiline

In [ ]:
print (
    f'F-strings provide a way to embed \'{"expressions"}\' inside string literals, '
    f'using a minimal syntax. '
)

### Ternary operator

In [ ]:
foo = None

f'{foo if foo is not None else "foo"}'

### Lambda functions

In [ ]:
f'{(lambda x: x*2)(3)}'

### List comprehensions

In [ ]:
celsius = [0, 20, 40]

[f'{1.8 * c + 32:.2f} Fahrenheit' for c in celsius]

### Handy formatting

**Nota**: Equivalente al de `format`.

In [ ]:
left = 'left'
center = 'center'
right = 'right'

f'{left:><15}{center:-^10}{right:<>15}'

In [ ]:
from math import pi

f'Pi: {pi} - {pi:.4f}'

### Bonus track: DEBUG  >= 3.8

```python
>>>foo = 30
>>> print(f'{foo=}  {cos(radians(foo))=:.3f}')
foo=30  cos(radians(foo))=0.866
```

In [ ]:
foo = 30

print(f'{foo=}  {cos(radians(foo))=:.3f}')

## Pitfalls

### Modern Python >= 3.6

En este caso, IMO esto es más una ventaja que un problema, a estas alturas de la película todos deberíamos estar al menos en esta versión de y evitar el *Legacy Python".

### Docstrings

Al evaluación en _runtime_ descarta la posibilidad de que las `F-strings` puedan usarse para documentar código.

### Quoting 

La sintaxis de las `F-strings` pueden resultar un tanto ardua en lo que se refiere al *entrecomillado*, de hecho existe una propuesta, la [PEP536](https://www.python.org/dev/peps/pep-0536/) que aboga por su modificación que entre otras cosas permitiría el uso de las comillas dentro de la expresión con independencia de las *exteriores* permitiendo expresiones del tipo: `f'Magic wand: {bag['wand']:^10}'`.

Esta `PEP` se encuentra en estado *Deferred*, es decir, no hay ningún desarrollador del core que se haya prestado voluntario a desarrollarla.

### Dicts

El uso con diccionarios puede resultar mucho mas *cómodo* con `format`:

In [ ]:
nerd = {'name': 'Juan Diego', 'from': 'Almería'}

print('This nerd is {name} from {from}'.format(**nerd))

print(f'This nerd is {nerd["name"]} from {nerd["from"]}')

### Logging

Al usar `F-strings` con logging podemos encontrarnos ante un problema de rendimiento debido a la llamada automática al método `__str__` del objeto:

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('fail')

class Dummy:
    def __init__(self, name):
        self._name = name
    def __str__(self):
        print('logging should be >= INFO')
        return self._name
    
c = Dummy('fstring')

logger.debug(f'Created: {c}')

En caso de que esto suponga un problema resulta más conveniente usar la interpolación tradicional:

In [ ]:
logger.debug('Created: %s', c)

### Last but not least

> There should be one-- and preferably only one --obvious way to do it.

Una de las mayores críticas a este nuevo sistemas es que no aporta nada nuevo que no pudiera hacerse con métodos como `format`, y que hace que el lenguaje sea cada vez más pesado.

# ¡¡ Gracias !!